In [2]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 아이콘을 클릭하고
# 옵션에서 기간 선택하기
# 블로그 글, 링크 수집하기

In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 크롤링 방법
# 1. requests, bs
# 2. v 셀레늄 
# 3. api 기반 

In [3]:
# 검색어
query_txt = "경기도 베이커리 카페"

In [2]:
# 크롬 드라이버 : 파이썬이 크롬 브라우저를 통제할 수 있게 도와주는 프로그램
# 크롬 드라이버를 '현재 작업 폴더'에 설치
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저
# driver : 크롬 브라우저

driver.get('https://www.naver.com/')
time.sleep(2)    # 2초간 정지 

In [4]:
# 네이버 검색창에 "검색어" 검색
selector = "#query"
element = driver.find_element(By.CSS_SELECTOR, selector)    # 검색창 찾기
element.clear()                # 현재 검색창에 무언가 들어있으면 삭제
element.send_keys(query_txt)   # 검색창에 검색어 입력

In [5]:
element.submit()    # 검색 버튼 클릭  
time.sleep(1)

In [ ]:
# a 태그에 포함된 텍스트는 LINK_TEXT를 이용해서 텍스트를 셀렉터로 사용할 수 있다.

In [6]:
# '블로그' 클릭
driver.find_element(By.LINK_TEXT, "블로그").click( )
time.sleep(1)

In [7]:
# '옵션' 클릭
driver.find_element(By.LINK_TEXT, "옵션").click( )
time.sleep(1)

In [8]:
# LINK_TEXT를 이용한 검색기간 '3개월' 클릭
driver.find_element(By.LINK_TEXT, "1주").click( )
time.sleep(1)

In [53]:
# # CSS SELCTOR를 이용한 "3개월" 클릭
# selector = "#snb > div.mod_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a:nth-child(6)"
# driver.find_element(By.CSS_SELECTOR, selector).click()
# time.sleep(1)

In [54]:
# # 현재 페이지를 스크롤 다운하는 자바스크립트 코드
# driver.execute_script("window.scrollTo(0, 99999999)")
# time.sleep(1)

In [11]:
# 글 개수 확인
def blog_cnt():
    # 글 CSS SELECTOR 
    selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
    blog_cnt = len(driver.find_elements(By.CSS_SELECTOR, selector))
    print(blog_cnt)
    
blog_cnt()

30


In [57]:
# 페이지 스크롤다운하는 코드
n = 3    # 스크롤할 횟수
i = 0    # 0, 1, 2
while i < n:
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)
    i = i + 1

In [58]:
blog_cnt()

120


# step1. 블로그 글 제목, url 수집

### 블로그 글 제목 수집

In [ ]:
# v3에서는 블로그 글 제목 selector로 class를 더블클릭 후 활용하였다.

In [12]:
# 첫번째 글 제목 수집
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
driver.find_element(By.CSS_SELECTOR, selector).text

'카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천'

In [37]:
# list(range(1, 61, 2))

In [14]:
# 글 제목 여러개를 수집 후 리스트로 저장
selector = ".sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1.sds-comps-text-weight-sm"
titles_raw = driver.find_elements(By.CSS_SELECTOR, selector)
# len(titles_raw)

title_list = []
for title_raw in titles_raw:
    title = title_raw.text
    title_list.append(title)
    # print(title)

print(len(title_list))
title_list

30


['카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천',
 '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천',
 '경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함',
 '[경기도 여주] 아이랑 가볼만한 유기농 밀가루 베이커리 카페, 가루영웅베이커리 방문 후기',
 '8개월 아기 안산 나들이 추천_경기도미술관·마리나베이커리·카페미스터신 수유실까지 완벽',
 '[경기도 포천 베이커리 카페]내촌상회',
 '경기도 양주 대형카페 브루다 양주 베이커리 카페',
 '아기랑 경기도 - 의정부 핫한 카페 투하츠 베이커리',
 '경기도 화성에서 찾은 숨겨진 보석, 고향의봄 베이커리 카페',
 '경기도 화성 카페: 고향의봄 베이커리 카페에서 만난 여유',
 '경치 맛집! 경기도 화성 카페, 고향의봄 베이커리 카페 방문 후기',
 '경기도 화성에서 찾은 숨은 보석, 고향의봄 베이커리 카페',
 '[시흥] 경기도 시흥 카페 베이커리가 맛있는 곳 "늘솜당" 서울, 인천근교 로 추천!',
 '경기도 화성에서 찾은 숨은 보석, 고향의봄 베이커리 카페 방문 후기!',
 '[경기도 성남/위례] 주차 가능한 남위례역 베이커리 카페 아우어 위례점',
 '경기도 화성에서 만난 초대형 베이커리 카페, 고향의봄 베이커리 카페 방문기',
 '경기도 건축문화상 은상 대형카페 페이브 베이커리 화성남양점',
 '경기도 화성에서 찾은 인생 쿠키 맛집, 고향의봄 베이커리 카페',
 '내돈내산 경기도 남양주 아기랑가기 좋은 베이커리카페 하우스 용암(haus yongam)',
 '경기도 화성에서 발견한 숨겨진 보석, 고향의봄 베이커리 카페!',
 '경기도 화성시 포근베이커리 카페/전통찻집 - 달콤한여행',
 '경기도 화성에서 맛있는 디저트 경험, 고향의봄 베이커리 카페',
 '경기도 화성에서 찾은 보물, 고향의봄 베이커리 카페 방문기',
 '경기도 하남시 감일지구 서하남근처 베이커리카페 온더브레드 하남점 브런치 맛집 모임추천 새벽까지 영업',
 '경

### 글 url 수집하기

In [ ]:
# .text : 태그 사이에 들어있는 텍스트 추출
# .get_attribute("href") : 태그 안에 들어있는 속성(ex. href url)값 추출

In [15]:
selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
title_raw = driver.find_element(By.CSS_SELECTOR, selector)
# len(titles_raw)
title_raw.get_attribute('href')

'https://blog.naver.com/doin1914/224164042092'

In [16]:
# 글 url 120개 수집하고 리스트로 저장하기
selector = ".fender-ui_228e3bd1.jc59IQoHJVaiFTctwPf1"
urls_raw = driver.find_elements(By.CSS_SELECTOR, selector)

url_list = []
for url_raw in urls_raw:   
    url = url_raw.get_attribute('href')
    url_list.append(url)
    # print(url)

print(len(url_list))
url_list

30


['https://blog.naver.com/doin1914/224164042092',
 'https://blog.naver.com/jung35745/224163886985',
 'https://blog.naver.com/oifkhlfzio935/224168098738',
 'https://blog.naver.com/dusendigd/224166414878',
 'https://blog.naver.com/sh_note106/224164508577',
 'https://blog.naver.com/comamiyo/224162292781',
 'https://blog.naver.com/sweatlatte46/224160792443',
 'https://blog.naver.com/nalmong126/224165872965',
 'https://blog.naver.com/pjuprhtdpr246/224164850015',
 'https://blog.naver.com/pvdwqbukac710/224166040543',
 'https://blog.naver.com/bemwrlgigc615/224167118923',
 'https://blog.naver.com/duezvaiyqs813/224167030929',
 'https://blog.naver.com/syeon_nolja/224164817653',
 'https://blog.naver.com/dgtshmoozq986/224168127447',
 'https://blog.naver.com/lucky_line_0507/224152689117',
 'https://blog.naver.com/mofdumcesb161/224167065030',
 'https://blog.naver.com/uquiz76ham/224164528406',
 'https://blog.naver.com/kqtczfypyh423/224163611543',
 'https://blog.naver.com/007topkorea/224160426819',
 'ht

In [19]:
# 데이터프레임 저장
df_title_url = pd.DataFrame({"글제목": title_list, "url":url_list })
df_title_url

,글제목,url
0,카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천,https://blog.naver.com/doin1914/224164042092
1,경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천,https://blog.naver.com/jung35745/224163886985
2,"경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함",https://blog.naver.com/oifkhlfzio935/224168098738
3,"[경기도 여주] 아이랑 가볼만한 유기농 밀가루 베이커리 카페, 가루영웅베이커리 방문 후기",https://blog.naver.com/dusendigd/224166414878
4,8개월 아기 안산 나들이 추천_경기도미술관·마리나베이커리·카페미스터신 수유실까지 완벽,https://blog.naver.com/sh_note106/224164508577
5,[경기도 포천 베이커리 카페]내촌상회,https://blog.naver.com/comamiyo/224162292781
6,경기도 양주 대형카페 브루다 양주 베이커리 카페,https://blog.naver.com/sweatlatte46/224160792443
7,아기랑 경기도 - 의정부 핫한 카페 투하츠 베이커리,https://blog.naver.com/nalmong126/224165872965
8,"경기도 화성에서 찾은 숨겨진 보석, 고향의봄 베이커리 카페",https://blog.naver.com/pjuprhtdpr246/224164850015
9,경기도 화성 카페: 고향의봄 베이커리 카페에서 만난 여유,https://blog.naver.com/pvdwqbukac710/224166040543


In [20]:

# csv로 저장
df_title_url.to_csv("title_url.csv", encoding='utf-8-sig', index=False)

In [ ]:
# [실습]
# 1. 글 class로 css selector 찾기 연습.
# 2. 검색어를 변경해서 글 url 200개 크롤링 및 데이터프레임, csv 저장.

# step2. 블로그 글 한 개 크롤링

In [65]:
# 이미 수집해둔 글 url로 접속해서 콘텐츠(글, 이미지 등) 수집 해보기

In [22]:
#shape 뜻: 모양, 형태
df_url = pd.read_csv("title_url.csv")

print(df_url.shape)
df_url.head()

(30, 2)


,글제목,url
0,카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천,https://blog.naver.com/doin1914/224164042092
1,경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천,https://blog.naver.com/jung35745/224163886985
2,"경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함",https://blog.naver.com/oifkhlfzio935/224168098738
3,"[경기도 여주] 아이랑 가볼만한 유기농 밀가루 베이커리 카페, 가루영웅베이커리 방문 후기",https://blog.naver.com/dusendigd/224166414878
4,8개월 아기 안산 나들이 추천_경기도미술관·마리나베이커리·카페미스터신 수유실까지 완벽,https://blog.naver.com/sh_note106/224164508577


### 첫번째 블로그 글 띄우기

In [23]:
# 첫번째 글 제목
df_url['글제목'][0]

'카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천'

In [24]:
# 첫번째 글의 url
i = 1
url = df_url['url'][i]
url

'https://blog.naver.com/jung35745/224163886985'

In [28]:
# 크롬 브라우저 띄우기
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1)

In [29]:
# iframe 접근 네이버블로그는 이거 꼭 써야한다고함 (✔ iframe 없는 페이지일 때일반 홈페이지네이버 검색 결과 페이지대부분 쇼핑몰 목록 페이지
#👉 “지금부터 작업 무대를 mainFrame으로 바꾼다” 👉 “방 밖에서 → 방 안으로 들어간다”
driver.switch_to.frame('mainFrame')

In [31]:
# 게시글(제목, 글쓴이, 날짜, 내용 등) 크롤링 후 저장할 딕셔너리({})
target_info = {}

# 블로그 글 제목 첫번째 크롤링
# selector = "#SE-308d4d8e-fccc-11f0-925e-8135ef4f9fb3"  # 첫시도
selector = ".se-module.se-module-text.se-title-text"     # 두번째 시도
title_raw = driver.find_element(By.CSS_SELECTOR, selector)          # title
title = title_raw.text
print(title)

# 딕셔너리에 데이터 저장
target_info['타이틀'] = title
target_info

경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천


{'타이틀': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천'}

In [32]:
# 글쓴이 크롤링
selector = ".nick"
nickname = driver.find_element(By.CSS_SELECTOR, selector).text
target_info['nickname'] = nickname   # 찾은 닉네임 데이터를 target_info에 저장 
target_info

{'타이틀': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천', 'nickname': '토행순\njung35745'}

In [34]:
# [실습] 날짜 크롤링
selector = ".se_publishDate.pcol2"
datetime = driver.find_element(By.CSS_SELECTOR, selector).text

# target_info 딕셔너리에 날짜 저장
target_info['시간'] = datetime
target_info

{'타이틀': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천',
 'nickname': '토행순\njung35745',
 'datetime': '2026. 1. 29. 12:17',
 '시간': '2026. 1. 29. 12:17'}

### 글 내용 수집 

In [37]:
# 예시로 한 문단 텍스트 확인  [0]첫텍스트에 공백이라서 아무 글있는 다섯번째 숫자 그냥 넣은거임
selector = ".se-component.se-text.se-l-default"
driver.find_elements(By.CSS_SELECTOR, selector)[5].text

# 반복문으로 모든 문단 텍스트 수집 후 리스트로 저장
review_str = ""

selector = ".se-component.se-text.se-l-default"
text_raw = driver.find_elements(By.CSS_SELECTOR, selector)

for review in text_raw:
    review_str = review_str + review.text.replace("\n", " ")
    # print(review.text)
review_str

target_info['review'] = review_str
target_info

{'타이틀': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천',
 'nickname': '토행순\njung35745',
 'datetime': '2026. 1. 29. 12:17',
 '시간': '2026. 1. 29. 12:17',
 'review': '안녕하세요. 토행순이에요. 요즘 날씨가 굉장히 춥잖아요. 그래서 저희도 데이트할 때 무조건 실내로 많이 알아보고 다니는데요. 그러다 고양시에 엄청 빵들이 맛있고 브런치로 유명한 대형베이커리카페를 찾아서 방문하게 되었어요. 바로 명장10이라고 하는 대형카페입니다!고양시에 위치한 명장10에 처음 방문했어요. 카페 앞에 주차장이 크게 있어서 주차는 무리 없었고 주중에는 1만원 결제하면 3시간 무료주차가 가능하고 주말이나 공휴일에는 2만원 결제하면 3시간 무료주차 가능! 근데 솔직히 이런 베이커리 카페가면 2만원 순삭이라.ㅎ얼른 카페 안으로 들어가봤는데 규모가 엄청 큽니다! 밖에서봐도 엄청 큰 카페인게 티가나는데 실제로 들어가보면 더 크게 느껴지고, 초록+우드 인테리어로 되어있어서 저는 굉장히 마음이 편했던 그런 곳이었어요.저희는 편한 자리에 자리잡고 호다닥 빵 고르러! 여기가 베이커리카페라서 진~짜 빵 종류가 엄청나게 많은거 있죠!!!! 빵덕후로써 이런거 굉장히 사랑합니다.거기다가 사장님께서 빵에 진심인게 느껴지는 것이 빵 나오는 시간이 종류마다 엄청 다른거에요. 만약 먹고싶은 빵이 있다면 맞춰서 방문하는 걸 추천해요. 요즘 두쫀쿠가 유행이라서 그런지 얘도 판매한대요!케이크 종류도 엄청 많이 있었는데요. 크리스마스 시즌이 되면 크리스마스 케이크를 따로 판매하고 계신다는 사실>< 심지어 사전예약하면 10%나 할인해주십니다.만약 빵 종류가 너무 많아서 뭘 먹을지 모르겠다 하시는 분들을 위해 Best10을 정해주셨어요. 바게트 샌드위치 / 몽블랑 / 소금빵 / 명장버거 바삭 밀크 프랑스 / 바질 토마토 / 구르메(브레첼) 마늘빵 / 바삭 앙버터 / 바삭 호두파이원래는 빵만 먹으려고 했는데! 여기

In [129]:
# (참고) 이미지 수집하기
selector = ".se-image-resource.egjs-visible"
images_raw = driver.find_elements(By.CSS_SELECTOR, selector)

# 첫번째 이미지
image = driver.find_elements(By.CSS_SELECTOR, selector)[0]
img_src = image.get_attribute('src')
img_src
# driver.get(img_src)

'https://postfiles.pstatic.net/MjAyNjAxMjlfNTcg/MDAxNzY5NjYwNTUxMDQy.k6qz7I8ICcS8aDO1c5HaVKpHjJqIfNQhPp6vIrdw7fAg.zr07Eu7VCwb4znJWKLMNogDcWhHg7dWmao_7AYtNDP8g.JPEG/900%EF%BC%BF1769660547811.jpg?type=w773'

In [130]:
# 첫번째 블로그 글 수집 완료!

# step3. 전체 블로그 글 크롤링

In [133]:
# 네이버 블로그 글 크롤링 프로세스
# 1. 각 블로그 글 링크 크롤링
# 2. 반복문으로 각 링크 돌면서 데이터(target_info) 수집 후 전체 딕셔너리(total_dict)에 저장.
# 3. 데이터프레임으로 저장. csv로 저장.

In [135]:
# (이런 형태로 저장)
total_dict = {}   # 전체 데이터를 담을 딕셔너리

total_dict[0] = {"title": "글1"}   # target_info 
total_dict[1] = {"title": "글2"}
# ...
total_dict[119] = {"title": "글120"}

total_dict

{0: {'title': '글1'}, 1: {'title': '글2'}, 119: {'title': '글120'}}

In [38]:
# 블로그 링크(url) 불러오기
df_url = pd.read_csv("title_url.csv")

print(len(df_url))
df_url.head(3)

30


,글제목,url
0,카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천,https://blog.naver.com/doin1914/224164042092
1,경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천,https://blog.naver.com/jung35745/224163886985
2,"경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함",https://blog.naver.com/oifkhlfzio935/224168098738


In [39]:
df_url['url']

0          https://blog.naver.com/doin1914/224164042092
1         https://blog.naver.com/jung35745/224163886985
2     https://blog.naver.com/oifkhlfzio935/224168098738
3         https://blog.naver.com/dusendigd/224166414878
4        https://blog.naver.com/sh_note106/224164508577
5          https://blog.naver.com/comamiyo/224162292781
6      https://blog.naver.com/sweatlatte46/224160792443
7        https://blog.naver.com/nalmong126/224165872965
8     https://blog.naver.com/pjuprhtdpr246/224164850015
9     https://blog.naver.com/pvdwqbukac710/224166040543
10    https://blog.naver.com/bemwrlgigc615/224167118923
11    https://blog.naver.com/duezvaiyqs813/224167030929
12      https://blog.naver.com/syeon_nolja/224164817653
13    https://blog.naver.com/dgtshmoozq986/224168127447
14    https://blog.naver.com/lucky_line_0507/2241526...
15    https://blog.naver.com/mofdumcesb161/224167065030
16       https://blog.naver.com/uquiz76ham/224164528406
17    https://blog.naver.com/kqtczfypyh423/22416

In [8]:
# # 블로그 글 하나 띄우고 target_info에 데이터 수집
# chromedriver_autoinstaller.install(cwd=True)
# driver = webdriver.Chrome()   # driver : 크롬 브라우저

# i = 0   # 첫번째 글 
# driver.get(df_url['url'][i]) 
# time.sleep(1)

# # iframe 접근
# driver.switch_to.frame('mainFrame')

# # 게시글(제목, 글쓴이, 날짜, 내용 등) 크롤링 후 저장할 딕셔너리({})
# target_info = {}

# # 블로그 글 제목 첫번째 크롤링
# selector = ".se-module.se-module-text.se-title-text"
# title_raw = driver.find_element(By.CSS_SELECTOR, selector)          # title
# title = title_raw.text
# print(title)

# # 딕셔너리에 데이터 저장
# target_info['title'] = title

# # 글쓴이 크롤링
# selector = ".nick"
# nickname = driver.find_element(By.CSS_SELECTOR, selector).text
# target_info['nickname'] = nickname   # 찾은 닉네임 데이터를 target_info에 저장 

# # 날짜 크롤링
# selector = ".se_publishDate.pcol2"
# datetime = driver.find_element(By.CSS_SELECTOR, selector).text
# # target_info 딕셔너리에 날짜 저장
# target_info['datetime'] = datetime

# # 반복문으로 모든 문단 텍스트 수집 후 리스트로 저장
# review_str = ""
# selector = ".se-component.se-text.se-l-default"
# text_raw = driver.find_elements(By.CSS_SELECTOR, selector)
# for review in text_raw:
#     review_str = review_str + review.text.replace("\n", " ")
#     # print(review.text)
# target_info['review'] = review_str

# target_info

In [40]:
# 블로그의 갯수
blog_cnt = len(df_url['url'])
blog_cnt

30

In [41]:
# for문으로 전체 블로그 글 크롤링
chromedriver_autoinstaller.install(cwd=True)

total_dict = {}  # 전체 블로그 글 수집할 딕셔너리 선언 

# i = 0   # 첫번째 글 
for i in range(0, 3):   # blog_cnt     
    try:    
        driver = webdriver.Chrome()   # driver : 크롬 브라우저
        driver.get(df_url['url'][i])  # i가 바뀌면서 블로그 글도 계속 바뀐다.
        time.sleep(1)
        
        # iframe 접근(네이버 사이트들의 특성 - 네이버 블로그, 네이버 카페)
        driver.switch_to.frame('mainFrame')
        
        # 한 개 블로그 글(제목, 글쓴이, 날짜, 내용 등) 크롤링 후 저장할 딕셔너리
        target_info = {}    # 선언, 초기화 
        
        # 블로그 글 제목 첫번째 크롤링
        selector = ".se-module.se-module-text.se-title-text"     
        title_raw = driver.find_element(By.CSS_SELECTOR, selector)          # title
        title = title_raw.text
        print(title)
        
        # 딕셔너리에 데이터 저장
        target_info['title'] = title
        
        # 글쓴이 크롤링
        selector = ".nick"
        nickname = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['nickname'] = nickname   # 찾은 닉네임 데이터를 target_info에 저장 
        
        # 날짜 크롤링
        selector = ".se_publishDate.pcol2"
        datetime = driver.find_element(By.CSS_SELECTOR, selector).text
        # target_info 딕셔너리에 날짜 저장
        target_info['datetime'] = datetime
        
        # 반복문으로 모든 문단 텍스트 수집 후 리스트로 저장
        review_str = ""
        selector = ".se-component.se-text.se-l-default"
        text_raw = driver.find_elements(By.CSS_SELECTOR, selector)
        for review in text_raw:
            review_str = review_str + review.text.replace("\n", " ")
            # print(review.text)
        target_info['review'] = review_str
    
        print(target_info)
        total_dict[i] = target_info   # i가 0~119까지 변하면서 각 블로그 글을 저장.
        print(i+1, "번째 데이터가 수집되었습니다.", "\n")

        time.sleep(2)

        driver.close()      # 블로그 글 한 개 마다 크롤링했던 브라우저 종료
        
    except:
        print(i+1, "번째 블로그는 건너 뜁니다.")
        driver.close()      # 크롬 브라우저 종료
        continue

카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천
{'title': '카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천', 'nickname': '대구 정기청소 해드림', 'datetime': '2026. 1. 29. 17:00', 'review': '안녕하세요~오늘은 경기도 안성시에 위치한 대형카페를 소개해 드릴게요~ 안성 분들 사이에는 이미 유명한 곳이더라구요. 바로 #카페델라고 입니다~ 위치는 이곳입니다.가게 사진을 보시죠!입구부터 여기가 카페야?? 할 정도로 규모가 커요. 주차장은 아주 넓으니 주차 걱정은 안 하셔도 됩니다~입구부터 여기 분위기가 너무 좋았는데 추워서 있을 수가 없었어요 ㅠㅠ 날 좋을 때 다시 와보고 싶네요.메뉴입니다~베이커리 메뉴가 다양해요~ 좀 늦은 시간에 방문해서 메뉴가 몇 가지 빠지긴 했지만 저녁에도 꽤 다양한 빵들이 있었어요!엘리베이터가 있으니 무거운 음료 들고 불안하게 계단으로 가지 마세요 ㅎㅎ2층으로 가보시겠습니다~~ 2층 내부 사진을 깜빡했네요;; 테이블이 엄청 많았구요. 이 시간에도 사람이 많았어요.테라스에 나가봤어요. 추운 날씨에 앉아 있진 못했지만 멍하게 저수지를 바라봤네요~ 조명에 비친 저수지가 너무 이쁘더라구요!한참 구경하고 왔더니 커피가 와 있었네요 ㅎㅎ 살짝 식었지만 그래도 이쁜 라테 아트~ 이 카페는 낮에 가도 참 예쁠 것 같아요. 다음에도 또 가고 싶어요. 안성에 가시면 꼭 들러보시길 바라요~#안성맛집 #안성카페추천 #안성원곡카페추천 #안성칠곡저수지카페추천 #안성대형베이커리카페추천 #안성칠곡리카페추천 #안성저수지뷰카페 #카페델라고 #안성여행 #안성대형카페추천'}
1 번째 데이터가 수집되었습니다. 

경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천
{'title': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천', 'nickname': '토행순\njung35745', 'datetime': '2026. 1

In [42]:
# 수집된 블로그 글 개수 확인
len(total_dict)
total_dict

{0: {'title': '카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천',
  'nickname': '대구 정기청소 해드림',
  'datetime': '2026. 1. 29. 17:00',
  'review': '안녕하세요~오늘은 경기도 안성시에 위치한 대형카페를 소개해 드릴게요~ 안성 분들 사이에는 이미 유명한 곳이더라구요. 바로 #카페델라고 입니다~ 위치는 이곳입니다.가게 사진을 보시죠!입구부터 여기가 카페야?? 할 정도로 규모가 커요. 주차장은 아주 넓으니 주차 걱정은 안 하셔도 됩니다~입구부터 여기 분위기가 너무 좋았는데 추워서 있을 수가 없었어요 ㅠㅠ 날 좋을 때 다시 와보고 싶네요.메뉴입니다~베이커리 메뉴가 다양해요~ 좀 늦은 시간에 방문해서 메뉴가 몇 가지 빠지긴 했지만 저녁에도 꽤 다양한 빵들이 있었어요!엘리베이터가 있으니 무거운 음료 들고 불안하게 계단으로 가지 마세요 ㅎㅎ2층으로 가보시겠습니다~~ 2층 내부 사진을 깜빡했네요;; 테이블이 엄청 많았구요. 이 시간에도 사람이 많았어요.테라스에 나가봤어요. 추운 날씨에 앉아 있진 못했지만 멍하게 저수지를 바라봤네요~ 조명에 비친 저수지가 너무 이쁘더라구요!한참 구경하고 왔더니 커피가 와 있었네요 ㅎㅎ 살짝 식었지만 그래도 이쁜 라테 아트~ 이 카페는 낮에 가도 참 예쁠 것 같아요. 다음에도 또 가고 싶어요. 안성에 가시면 꼭 들러보시길 바라요~#안성맛집 #안성카페추천 #안성원곡카페추천 #안성칠곡저수지카페추천 #안성대형베이커리카페추천 #안성칠곡리카페추천 #안성저수지뷰카페 #카페델라고 #안성여행 #안성대형카페추천'},
 1: {'title': '경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천',
  'nickname': '토행순\njung35745',
  'datetime': '2026. 1. 29. 12:17',
  'review': '안녕하세요. 토행순이에요. 요즘 날씨가 굉장히 춥잖아요. 그래서 저희도 데이트할 때 무조건 실내로 많이 알

In [43]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(total_dict, 'index')   # 'index' : 행열 변환
result_df

,title,nickname,datetime,review
0,카페델라고 - 경기도 안성시 원곡면 만세로 칠곡저수지뷰 대형 베이커리 카페 추천,대구 정기청소 해드림,2026. 1. 29. 17:00,안녕하세요~오늘은 경기도 안성시에 위치한 대형카페를 소개해 드릴게요~ 안성 분들 사...
1,경기도 당일치기 고양 두쫀쿠 대형베이커리카페 명장10 브런치카페 추천,토행순\njung35745,2026. 1. 29. 12:17,안녕하세요. 토행순이에요. 요즘 날씨가 굉장히 춥잖아요. 그래서 저희도 데이트할 때...
2,"경기도 화성 카페, 고향의봄 베이커리 카페에서 찾은 특별함",구름사다리,15시간 전,"안녕하세요. 오늘은 제가 최근에 다녀온 경기도 화성 카페, '고향의봄 베이커리 카페..."


In [42]:
# result_df.to_csv("네이버 블로그_경기도 베이커리 카페_10개.csv", encoding="utf-8-sig", index=False)

In [44]:
result_df.to_excel("네이버 블로그_경기도 베이커리 카페_10개.xlsx", index=False)